In this notebook, we show how to run RSVP on all 44 datasets.

Make sure you have the `Matrix` R package installed.

For the analysis, we keep those genes that are : 1) protein coding; 2) autosomal; 3) are annotated by the gene ontology.

When considering gene-gene edges, we look at **inter-chromosomal** genes only (to avoid problems due to linkage disequilibrium).

# Loading the data


In [4]:
rm(list=ls()) # Clear the environment

# Loading the expression matrices of all tissues. 
load("./DataPreparation/all_tissues.preprocessed.data.RData.gz")
# You should have two variables : datasets and ens_to_symbol. 
ls()

[1] "datasets"      "ens_to_symbol"

The `datasets` variable has one key per tissue:

In [6]:
names(datasets)

[1] "Adipose_Subcutaneous"                 
 [2] "Adipose_Visceral_Omentum"             
 [3] "Adrenal_Gland"                        
 [4] "Artery_Aorta"                         
 [5] "Artery_Coronary"                      
 [6] "Artery_Tibial"                        
 [7] "Brain_Anterior_cingulate_cortex_BA24" 
 [8] "Brain_Caudate_basal_ganglia"          
 [9] "Brain_Cerebellar_Hemisphere"          
[10] "Brain_Cerebellum"                     
[11] "Brain_Cortex"                         
[12] "Brain_Frontal_Cortex_BA9"             
[13] "Brain_Hippocampus"                    
[14] "Brain_Hypothalamus"                   
[15] "Brain_Nucleus_accumbens_basal_ganglia"
[16] "Brain_Putamen_basal_ganglia"          
[17] "Breast_Mammary_Tissue"                
[18] "Cells_EBV-transformed_lymphocytes"    
[19] "Cells_Transformed_fibroblasts"        
[20] "Colon_Sigmoid"                        
[21] "Colon_Transverse"                     
[22] "Esophagus_Gastroesophageal_Junction"  
[23] "Esophagus_Mucosa"                     
[24] "Esophagus_Muscularis"                 
[25] "Heart_Atrial_Appendage"               
[26] "Heart_Left_Ventricle"                 
[27] "Liver"                                
[28] "Lung"                                 
[29] "Muscle_Skeletal"                      
[30] "Nerve_Tibial"                         
[31] "Ovary"                                
[32] "Pancreas"                             
[33] "Pituitary"                            
[34] "Prostate"                             
[35] "Skin_Not_Sun_Exposed_Suprapubic"      
[36] "Skin_Sun_Exposed_Lower_leg"           
[37] "Small_Intestine_Terminal_Ileum"       
[38] "Spleen"                               
[39] "Stomach"                              
[40] "Testis"                               
[41] "Thyroid"                              
[42] "Uterus"                               
[43] "Vagina"                               
[44] "Whole_Blood"

For each tissue there are 4 entries:
-  covariates : a $q \times n$ matrix of covariates. There is one row per confounder and one column per sample.
-  expr.normal : a $n \times p$ matrix of gene expressions. There is one row per sample and one column / gene.
-  expr.unconfounded: a $n \times p$ matrix of gene expressions. It is the matrix of residuals from the regression `lm(expr.norm ~ t(covariates))`. 
-  chromosomes: a vector of size $p$, the chromosomes of the $p$ genes.

# R code to estimate a graph with RSVP

In [43]:
library(Matrix)
estimate.graph.with.RSVP <- function(X, chrs, n.edges=100) {
    p <- ncol(X)
    n <- nrow(X)

    X <- scale(X, scale=FALSE)
    
    # Return V V^T
    coefs <- tcrossprod(svd(scale(X, scale=FALSE), nu=0, nv = nrow(X)-1)$v)
    colnames(coefs) <- rownames(coefs) <- colnames(X)

    # Keep only inter chromosomal edges
    for (i in 1:p) {
        y_chr <- chrs[i]
        idx <- which(chrs == y_chr)
        coefs[i, idx] <- 0
    }
    
   coefs[lower.tri(coefs)] <- 0
    # Keep the top 2*n edges
    th <- sort(abs(coefs), decreasing = T)[(n.edges)] # Cutoff value
    coefs[abs(coefs) < th] <- 0
    coefs <- Matrix(coefs, sparse = T)
    nnz <- which(coefs !=0 , arr.ind = T)
    nnz2 <- which(coefs !=0)
    nnz <- cbind(nnz, coefs[nnz2])
    path <- nnz[order(abs(nnz[,3]), decreasing = T),]

    # A matrix with three columns : Edges 1 -- Edges 2 -- Value of the coefficient
    path
}

Warning message:
“package ‘Matrix’ was built under R version 3.4.4”

# Running RSVP on all tissues

We can now loop through the tissues and compute a graph on the raw datasets. Running RSVP should take around 2 minutes per tissue. 

In [55]:
valid_genes <- read.csv("./DataPreparation/ensembl_genes_with_go_annotations.txt", header=FALSE)
tissues <- names(datasets)
dir.create(file.path("./", "results")) # Create a directory for the results

count <- 0
for (tissue in tissues) {
    chrs <- get(tissue, datasets)$chromosomes

    X <- get(tissue, datasets)$expr.normal
    # Keep only the genes for which we have known annotations
    idx <- which(colnames(X) %in% valid_genes$V1)
    X <- X[,idx]
    chrs <- chrs[idx]
    print(paste("Tissue:", tissue, ", Dimensions:", nrow(X), "x", ncol(X)))
    start <- Sys.time()
    path <- estimate.graph.with.RSVP(X = X, chrs = chrs)
    stop <- Sys.time()
    print(stop - start)
    
    # Save the path to a csv file so we can analyse it later.
    fn <- paste("./results/", tissue, "_raw_dataset.csv", sep="")
    write.csv(path, file=fn)
    
    # Here we stop after two tissues
    count <- count + 1
    if (count >= 2) {
        break()
    }
}

Warning message in dir.create(file.path("./", "results")):
“'.//results' already exists”

[1] "Tissue: Adipose_Subcutaneous , Dimensions: 385 x 14585"
Time difference of 2.033186 mins
[1] "Tissue: Adipose_Visceral_Omentum , Dimensions: 313 x 14658"
Time difference of 1.649907 mins


# Analysing the results

The analysis of the graphs is done in Python3 in another notebook. See the README.

In [105]:
# This creates .csv files that contain the list of genes used for each tissue
row.names(nms) <- tissues
colnames(nms) <- valid_genes$V1
for (tissue in names(datasets)) {
    X <- get(tissue, datasets)$expr.normal
    cnames <- colnames(X)
    idx <- which(colnames(X) %in% valid_genes$V1)
    X <- X[,idx]
    cnms <- colnames(X)
    fn <- paste('./DataPreparation/valid_genes_per_tissue/', tissue, '.csv', sep="")
    write.csv(cnms, file=fn)
}